# Preparación del modelo

## Importación de librerías

In [1]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
import seaborn as sns

# Carga de datos

## Geodataframe

In [12]:
agebs = gpd.read_file('data/colima/agebs.gpkg')
agebs["area"] = agebs.area / 1000 # ha
agebs.drop(columns=["CVE_ENT","CVE_MUN", "CVE_AGEB","CVE_LOC","geometry"], inplace=True)
agebs = pd.DataFrame(agebs)

## Zonal Statistics

In [14]:
df = pd.read_csv('data/megaGDF.csv')
df = df.merge(agebs, on="CVEGEO")
df.drop(columns=["CVE_ENT","CVE_MUN","CVE_LOC"], inplace=True)
df.head()

,CVEGEO,CVE_AGEB,_count,_sum,_mean,class,time,area
0,0600200010676,0676,0.0,0.000000,NaN,Depth,0,381.991619
1,0600200010676,0676,2918.0,1344.881608,0.460892,Depth,10200,381.991619
2,0600200010676,0676,2922.0,1347.627452,0.461200,Depth,10500,381.991619
3,0600200010676,0676,2923.0,1349.168680,0.461570,Depth,10800,381.991619
4,0600200010676,0676,2924.0,1350.054582,0.461715,Depth,11100,381.991619


## Denue and INV

In [4]:
denue_inv = pd.read_csv('data/denue_inv.csv')
denue_inv.head()

,CVE_AGEB,POBTOT,POB0_14_P,P15A29A_P,P30A59A_P,GRAPROES,P_CD_P,VPH_C_EL_P,VPH_EXSA_P,VPH_DREN_P,...,PARATRAN_C,DRENAJEP_C,TRANSCOL_C,ARBOLES_C,ACESOAUT_C,PUESSEMI_C,PUESAMBU_C,escuela,farmacia,hospital
0,0273,99,107.54,6.137333,15.728000,3.978667,3.346667,33.333333,32.222000,33.333333,...,54,54,53,46,54,54,54,1.0,0.0,0.0
1,0318,1959,582.91,17.921154,32.048654,10.256538,7.542500,86.151731,86.311923,86.311923,...,164,149,136,102,165,167,164,13.0,6.0,0.0
2,0322,2143,617.33,17.747170,30.118113,11.442453,7.085472,88.486038,88.553396,88.553396,...,155,150,144,70,159,155,151,13.0,3.0,3.0
3,0362,2342,644.78,21.292000,33.820000,9.617714,5.100857,91.145714,91.380857,91.380857,...,103,102,97,45,105,102,102,4.0,1.0,1.0
4,0411,1140,381.58,17.902195,33.179268,11.795610,4.030732,80.487805,80.487805,80.487805,...,134,133,130,87,68,134,132,12.0,2.0,0.0


## Join

In [15]:
df_full = df.merge(denue_inv, on="CVE_AGEB")
df_full

,CVEGEO,CVE_AGEB,_count,_sum,_mean,class,time,area,POBTOT,POB0_14_P,...,PARATRAN_C,DRENAJEP_C,TRANSCOL_C,ARBOLES_C,ACESOAUT_C,PUESSEMI_C,PUESAMBU_C,escuela,farmacia,hospital
0,0600200010676,0676,0.0,0.000000,NaN,Depth,0,381.991619,2119,587.58,...,91,84,87,45,93,88,91,9.0,1.0,0.0
1,0600200010676,0676,2918.0,1344.881608,0.460892,Depth,10200,381.991619,2119,587.58,...,91,84,87,45,93,88,91,9.0,1.0,0.0
2,0600200010676,0676,2922.0,1347.627452,0.461200,Depth,10500,381.991619,2119,587.58,...,91,84,87,45,93,88,91,9.0,1.0,0.0
3,0600200010676,0676,2923.0,1349.168680,0.461570,Depth,10800,381.991619,2119,587.58,...,91,84,87,45,93,88,91,9.0,1.0,0.0
4,0600200010676,0676,2924.0,1350.054582,0.461715,Depth,11100,381.991619,2119,587.58,...,91,84,87,45,93,88,91,9.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6556,0601000010362,0362,0.0,0.000000,NaN,Velocity,900,227.156848,2342,644.78,...,103,102,97,45,105,102,102,4.0,1.0,1.0
6557,0601000010362,0362,0.0,0.000000,NaN,Velocity,9000,227.156848,2342,644.78,...,103,102,97,45,105,102,102,4.0,1.0,1.0
6558,0601000010362,0362,0.0,0.000000,NaN,Velocity,9300,227.156848,2342,644.78,...,103,102,97,45,105,102,102,4.0,1.0,1.0
6559,0601000010362,0362,0.0,0.000000,NaN,Velocity,9600,227.156848,2342,644.78,...,103,102,97,45,105,102,102,4.0,1.0,1.0


In [16]:
df_full.describe()

,_count,_sum,_mean,time,area,POBTOT,POB0_14_P,P15A29A_P,P30A59A_P,GRAPROES,...,PARATRAN_C,DRENAJEP_C,TRANSCOL_C,ARBOLES_C,ACESOAUT_C,PUESSEMI_C,PUESAMBU_C,escuela,farmacia,hospital
count,6318.000000,6.318000e+03,5080.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,...,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000
mean,7897.458371,2.935569e+04,2.452911,12046.913580,399.982035,1770.259259,520.540741,18.291919,32.611740,10.222105,...,112.074074,106.851852,101.925926,61.444444,106.259259,112.666667,111.037037,7.814815,2.666667,0.888889
std,7277.629876,8.085077e+04,5.092213,7058.657977,123.103015,817.774515,249.387994,3.559731,4.771108,1.920535,...,28.738444,29.786312,26.435616,19.620710,31.312771,28.829910,28.188672,5.994510,2.789079,1.165432
min,0.000000,0.000000e+00,0.000000,0.000000,201.591953,99.000000,107.540000,6.137333,15.728000,3.978667,...,54.000000,54.000000,53.000000,33.000000,52.000000,54.000000,54.000000,1.000000,0.000000,0.000000
25%,484.750000,6.338227e+01,0.220432,6000.000000,315.090313,1258.000000,298.180000,16.823333,29.926604,9.434516,...,91.000000,82.000000,86.000000,45.000000,81.000000,88.000000,90.000000,3.000000,1.000000,0.000000
50%,6977.000000,1.946393e+03,0.720000,12000.000000,407.209309,1935.000000,502.610000,18.112558,33.179268,10.188958,...,112.000000,108.000000,101.000000,58.000000,102.000000,114.000000,111.000000,6.000000,2.000000,1.000000
75%,13714.000000,1.645783e+04,1.860000,18300.000000,457.692900,2143.000000,617.330000,20.527500,35.534118,11.442453,...,134.000000,133.000000,123.000000,71.000000,129.000000,134.000000,132.000000,12.000000,3.000000,1.000000
max,28751.000000,1.043661e+06,36.300000,24300.000000,715.165608,4397.000000,1341.770000,25.398293,40.178511,13.536364,...,164.000000,159.000000,151.000000,112.000000,165.000000,167.000000,164.000000,26.000000,14.000000,4.000000
